# Relatório Parcial : Maratona de Filmes

Alunos: Bernardo Cunha Capoferri, Lívia Sayuri Makuta.



## Índice :
* [Objetivo do projeto](#first-bullet)
* [Detalhe de implementação da heurística gulosa e aleatória](#second-bullet)
* [Considerações sobre o profiling](#third-bullet)
* [Comparação entre as heurísticas conforme muda o valor](#fourth-bullet)
* [](#fifth-bullet)

O que você deverá fazer:

No blackboard, você deve fazer upload de todos os códigos-fonte, arquivos de input, arquivos de output para cada heurítica. Caso opte por enviar um link do github com o repositório completo, também poderá faze-lo, desde que garanta que teremos acesso aos arquivos no seu repositório;

Você deve elaborar um relatório parcial contendo as seguintes seções:

Para cada heurística você deve explicar como implementou a heurística (detalhe como você tratou o input, qual a lógica do seu output, quais invariantes existem em suas heurísticas), apresentar (i) o código-fonte comentado, (ii) fazer considerações sobre o profiling (valgrind) do código-fonte (use apenas 1 arquivo de input para isso, não há necessidade de fazer esse profiling para vários inputs), (iii) o resultado compartivo entre as heurísticas quando você varia o input (o input deve variar na quantidade de filmes e de categorias).

Seu relatório deve ser gráficos e tabelas que subsidem as suas considerações

É permitido criar um programa em python ou outra linguagem que automatize a geração de seus resultados, isto é, que execute seus códigos C++ em função dos diferentes inputs.

48 segundos: heuristica gulosa com 100000000
    
53 segundos : aleatorização com 100000000

54 segundos misturado com 100000000.


e 400 categorias


# Objetivo do projeto <a class="anchor" id="first-bullet"></a>

O projeto parte do seguinte cenário: em um final de semana você quer assistir o máximo de filmes posssíveis. Entretanto, para que isso aconteça, existem algumas restrições de horário e de categorias. Isso implica nas seguintes situações: o usuário pode assistir apenas um filme durante um intervalo de tempo, e precisa atender a quantidade de filmes por categoria que são desejados. Isso em forma de dados equivaleria ao seguinte:

- Como entrada temos uma quantidade de filmes `n` inteiros disponíveis;
- Além disso, também temos uma quantidade `m` inteiros que representa o número máximo de filmes que podem ser assistidos em cada categoria (comédia, drama, ação, etc).
- Por fim, temos 'n' trios de inteiros que representam a hora de início, a hora de fim e a categoria do filmes: `H[i], F[i], C[i]`, respectivamente.


Dessa forma, o formato de dados de entrada seguiria o seguinte exemplo:

``` txt
10 4
1 3 1 2 
11 13 3
14 15 3
10 16 2
10 14 1
11 17 2
11 14 3
13 15 3
14 15 1
12 16 4
12 13 4
```

Sendo nesse caso:

- `n = 10`, ou seja, existem 10 filmes disponíveis para serem assitidos.
- `m = 1, 3, 1, 2`, ou seja, podem ser assistidos: 1 filme para a categoria 1, 3 filmes para a categoria 2, 1 filme para a categoria 3 e 2 filmes para a categoria 4.
- `(H[1], F[1], C[1]) = (11, 13, 3); (H[2], F[2], C[2]) = (14, 15, 3)` e assim por diante. Ou seja, o filme 1 começa às 11h, termina às 13h e pertence a categoria 3, enquanto o filme 2 começa às 14, termina às 15h e pertence a categoria 3.

Logo, o objetivo do projeto é conseguir agrupar o máximo de filmes que podem ser assistidos em um dia e seguindo as restrições impostas. assim, como saída de dados nosso programa deve retornar: um inteiro que representa o número máximo dos filmes e os filmes que podem ser assistidos. Exemplo:

```txt
Foram vistos 2 filmes.
12 13 4
14 15 1
```

# Detalhe de implementação da heurística gulosa e aleatória <a class="anchor" id="second-bullet"></a>

Neste tópico será explicado como as heurísticas foram implementadas no nosso código em C++. Mas antes, importante ressaltar que todos os dados de entrada e arquivos no formato `.txt`foram gerados pelo código de exemplo passado pelo professor no site da disciplina: https://insper.github.io/supercomp/projetos/ .

## Heurística gulosa

Antes de explicar como essa heurística foi implementada, primeiro começaremos explicando seu conceito e como ela funciona. Dessa forma, tem-se que a ideia principal da heurística gulosa (Greedy) é tentar encontrar uma solução  global ótima. E para fazer isso, seguimos a seguinte abordagem:


1) Primeiro escolhemos uma solução inicial. No caso, escolhemos ordenar os filmes em ordem crescente do horário de fim. Depois, com eles ordenados, escolhemos ordenar novamente apenas aqueles filmes cujos horários de fim são os mesmos. E para esses, aplicamos uma ordenção crescente em relação ao horário de início.

2) Depois, aplicamos as restrições e escolhemos o primeiro filme que as atendia de acordo com os horários de fim que estão organizado de maneira crescente.

Logo, como pode ser visto, o objetivo da heurística gulosa é encontrar a solução ótima em um problema de otimização. No entanto, nem sempre é possível garantir que a solução encontrada seja a melhor possível, pois pode haver soluções melhores que não foram consideradas devido à abordagem local do algoritmo.

Apesar disso, a heurística gulosa é muito utilizada em problemas de otimização, especialmente quando o tamanho do problema é grande e não é possível encontrar a solução ótima de forma eficiente. Além disso, o algoritmo pode ser modificado e combinado com outras técnicas - como faremos posteriormente - para melhorar a qualidade da solução encontrada.


### Implementação do código

O primeiro passo para implementar a heurística gulosa foi limpar os dados de entrada e prepará-los para serem utilizados. Dessa maneira, foi criada uma struct Filmes para organizar melhor cada dado de entrada que representa um filme no seguinte formato: H[i], F[i], C[i]. Assim, essa struct `Filmes` foi construída recebendo o horário de início, o horário de fim e a categoria do filme, como pode ser visto abaixo:

``` cpp

struct Filme{
    int inicio;
    int fim;
    int categoria;
};

```

Feito isso, foram criadas as variáveis referentes à quantidade de filmes e categorias, bem como um vetor em que cada índice continha a quantidade de filmes que poderiam ser assistidos para cada categoria. **Detalhe importante:** essa última definição vai implicar no fato de que a categoria não corresponde diretamente ao índice do vetor, isso porque o índice do vetor começa em 0, enquanto a categoria de filmes começa em 1, logo, toda vez que quisermos acessar o número de filmes por categoria, devemos consultar o vetor em relação a uma categoria, precisaremos subtrair 1. 


O código correspondente ao que foi descrito foi implementado da seguinte maneira:

``` cpp

    int qtd_filmes, qtd_categorias;
    cin >> qtd_filmes >> qtd_categorias;

    vector<int> filmes_por_categoria(qtd_categorias, 0);

```

O código utilizou a função `cin`, que pertence a biblioteca iostream, e que recebe o dado de entrada do arquivo na ordem em que ele é lido, sendo assim, como os inputs começam com `n` (número de filmes disponíveis) e `m` (número de categorias disponíveis), primeiro foi preenchida uma variável que chamamos de `qtd_filmes` e depois outra variável que chamamos de `qtd_categorias`. 

Além disso, note que o vetor `filmes_por_categoria` foi preenchido com zeros, e isso porque nós temos a informação de seu tamanho - que é a quantidade de categorias. Isso é melhor do que usar a função `push_back()` por dois motivos:

- Quando `push_back()` é utilizado para adicionar novos elementos ao vetor, o compilador precisa alocar memória dinamicamente, e isso pode levar mais tempo - já que o sistema operacional precisa localizar um espaço livre de memória para conseguir alocar o elemento. Por sua vez, quando o vetor já é alocado com zeros desde o início, o compilador já aloca toda a memória de uma vez.

- Toda vez que o `push_back()` é chamado, uma chamada de função é feita, e isso em termos de tempo de execução pode ser caro - principalmente se muitos elementos forem adicionados ao vetor. Em contrapartida, se desde o início ele for preenchido com zeros, não há a necessidade de chamar a função `push_back()` repetidamente.


Com o vetor de filmes por categoria criado e já preenchido com zeros, o próximo passo foi substituir esses zeros com os dados de cada categoria. Mais uma vez, levando em consideração os dados de entrada e a sequência em que foram apresentados, foi utilizada a função `cin`, como pode ser visto abaixo:

``` cpp

   for (int i = 0; i < qtd_categorias; i++){
        cin >> filmes_por_categoria[i];
    }

```

Feito isso, para que o acesso a todos os dados de entrada fosse concluído, criamos uma variável chamada `matriz_filmes`, que é um vetor de `Filmes` (struct) 



